In [1]:
"""
Image Stitching Problem
(Due date: Nov. 9, 11:59 P.M., 2020)
The goal of this task is to stitch two images of overlap into one image.
To this end, you need to find feature points of interest in one image, and then find
the corresponding ones in another image. After this, you can simply stitch the two images
by aligning the matched feature points.
For simplicity, the input two images are only clipped along the horizontal direction, which
means you only need to find the corresponding features in the same rows to achieve image stiching.

Do NOT modify the code provided to you.
You are allowed use APIs provided by numpy and opencv, except “cv2.findHomography()” and
APIs that have “stitch”, “Stitch”, “match” or “Match” in their names, e.g., “cv2.BFMatcher()” and
“cv2.Stitcher.create()”.
"""
import cv2
import numpy as np
import random

def solution(left_img, right_img):
    """
    :param left_img:
    :param right_img:
    :return: you need to return the result image which is stitched by left_img and right_img
    """

    #sift initialization - Scale Invariant features
    sift   = cv2.SIFT_create()
    
    #Left Image Key Points
    leftimg= cv2.cvtColor(left_img,cv2.COLOR_BGR2GRAY)
    leftkeypnts= sift.detect(leftimg,None)
    
    #Right Image Key Points
    rightimg= cv2.cvtColor(right_img,cv2.COLOR_BGR2GRAY)
    rightkeypnts= sift.detect(rightimg,None)
    
    #Extract key point features descriptors for both left and right image
    #reducing search area of match points in left and right images
    leftsrcharea=len(leftkeypnts)//2     
    rightsrcharea=len(rightkeypnts)//2   
    
    leftkeypnts,leftfkdes=sift.compute(leftimg,leftkeypnts[leftsrcharea:])
    rightkeypnts,rightfkdes=sift.compute(rightimg,rightkeypnts[:rightsrcharea])
    
    #######################################Coded Matching Algorithm######################################## 
    ratio=0.85  #Lowe's Ratio Test to identify good matching points
    kNN=2
    keymatches=[]
    
    #Calculating euclidean distance(l2 norm) from each point in left to all points in right image
    #Find best matching point having lowest distance
    
    dist, nidx=cv2.batchDistance(leftfkdes, rightfkdes, K=kNN, dtype=-1, normType = cv2.NORM_L2)
    

    for ld in range(len(leftkeypnts)):   
        matchperpt=[]
        for i in range(kNN):
            matchperpt.append([ld,nidx[ld,i],dist[ld,i]])
            
        keymatches.append(matchperpt)
        
    best_match = []

    for m1, m2 in keymatches:
        if m1[2] < ratio * m2[2]:
            best_match.append((m1[0], m1[1]))

    #####################################################################################################
    
    ###################################Finding Homography Matrix#########################################
    #To find homography matrix we need 4 good matching points
    
    MIN_MATCH_COUNT = 4
    RANSAC_THRESH=5.0

    if len(best_match) > MIN_MATCH_COUNT:
        src_pts = np.float32([ leftkeypnts[m[0]].pt for m in best_match ]).reshape(-1,1,2)
        dst_pts = np.float32([ rightkeypnts[m[1]].pt for m in best_match ]).reshape(-1,1,2)
        
        H, mask = cv2.findHomography(dst_pts,src_pts, cv2.RANSAC, RANSAC_THRESH)
    
    ###################################################################################################
    #Wrap Perspective - to identify the position wrt to overlap part
    #Stitch by joining the images handling the overlap part using wrap perspective
    #eliminate excess points
    
        left_height= left_img.shape[0]
        left_width = left_img.shape[1]
        right_width= right_img.shape[1]
        new_height = left_height
        new_width  = left_width + right_width
    
        result = cv2.warpPerspective(right_img, H, (new_width, new_height))
        result[0:left_height,0:left_width]=left_img
        
        #Eliminating the excess ends
        for i in range(new_width-1,0,-1):
            if len(np.unique(result[:,i]))==1 and np.unique(result[:,i])==0:
                result=np.delete(result,i,axis=1)
            else:
                break
                
        
        return result
  
    ####################################################################################################
    

if __name__ == "__main__":
    left_img = cv2.imread('left.jpg')
    right_img = cv2.imread('right.jpg')
    result_image = solution(left_img, right_img)
    cv2.imwrite('results/Panorama1.jpg',result_image)
    
    
    left_img = cv2.imread('task1_sample_left.jpg')
    right_img = cv2.imread('task1_sample_right.jpg')
    result_image = solution(left_img, right_img)
    cv2.imwrite('results/Panorama2.jpg',result_image)

In [2]:
import cv2
import numpy as np
import math

def Hline_accum(img,p_res=1,theta_res=1):
    #Hough Line Accumulator is voting based algorithm 
    #We transform the image to the parameter space for line it just two parameters 
    #xcos(theta) + ysin(theta) = p
    #Increment the cell if point which has the same p and theta values
    #the one with more votes(more pixels) is the line with theta and p 
    
    h,w=img.shape
    
    #Calculate image diagonal using the pythagoras theorem
    diag=np.ceil(np.sqrt(h**2+w**2))
    
    #creating a parameter space with axis as p and theta
    ps= np.arange(-diag, diag + 1, p_res)
    thetas = np.deg2rad(np.arange(-90, 90, theta_res))
    
    #Hough Accumulator - A matrix to track the votes
    Hough  = np.zeros((len(ps), len(thetas)), dtype=np.uint64)
    xs, ys = np.nonzero(img)   
    
    
    #Find each edge point and accumulate votes
    for e in range(len(xs)):
        x=xs[e]
        y=ys[e]
        
        for t in range(len(thetas)):
            p = int((x * np.cos(thetas[t]) + y * np.sin(thetas[t])) + diag)
    
            Hough[p,t] +=1 #voting
        
    return Hough,ps,thetas

def getHLines(H, nhlines, thresh=0):
    #This method allows to select the lines with more number of votes if threshold parameter is 0
    #if threshold parameter has value then it selects only lines less than that threshold
    
    Hbackup = np.copy(H)
    nlinesidx = []
    
    for i in range(nhlines):
        idx = np.argmax(Hbackup)
        line_idx = np.unravel_index(idx, Hbackup.shape) 
        votes=Hbackup[line_idx]
        
        if thresh != 0:
            while votes>thresh:
                Hbackup[line_idx]=0
                idx = np.argmax(Hbackup)
                line_idx = np.unravel_index(idx, Hbackup.shape) 
                votes=Hbackup[line_idx]
        
        Hbackup[line_idx]=0
        nlinesidx.append(line_idx)
        
    return nlinesidx


def drawHLines(img, hLines, p, thetas, color,ignoreidx,filename):
    
    with open (filename, 'w') as filehandler:
        for i in range(len(hLines)):
            if len(ignoreidx)!=0:
                if i not in ignoreidx:  
                    rho = p[hLines[i][0]]
                    theta = thetas[hLines[i][1]]

                    filehandler.write(str([np.degrees(theta),rho])+'\n')
                    a = np.cos(theta)
                    b = np.sin(theta)
                    x0 = a*rho
                    y0 = b*rho

                    x1 = int(x0 + 1000*(-b))
                    y1 = int(y0 + 1000*(a))
                    x2 = int(x0 - 1000*(-b))
                    y2 = int(y0 - 1000*(a))

                    cv2.line(img, (y1, x1), (y2, x2), color, 2)

            else:
                rho = p[hLines[i][0]]
                theta = thetas[hLines[i][1]]

                filehandler.write(str([np.degrees(theta),rho])+'\n')

                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a*rho
                y0 = b*rho

                x1 = int(x0 + 1000*(-b))
                y1 = int(y0 + 1000*(a))
                x2 = int(x0 - 1000*(-b))
                y2 = int(y0 - 1000*(a))

                cv2.line(img, (y1, x1), (y2, x2), color, 5)
            
def removerlaps(linelist,H,mindist=0):
    final_list=[]    
    distance=[]

    for i in range(len(linelist)):
        dist=[]
        for j in range(len(linelist)):
            if i == j:
                dist.append(99999)  #set max distance for the same node 
            else:
                dist.append(cv2.norm(np.array(linelist[i]),np.array(linelist[j]), normType=cv2.NORM_L2))
                            
        
        distance.append([dist[np.argmin(dist)],np.argmin(dist)])
    
    for i in range(len(distance)):
        if distance[i][0]<mindist:
            if H[linelist[i]]>H[linelist[distance[i][1]]]:
                final_list.append(linelist[i])
            else:
                final_list.append(linelist[distance[i][1]])
        else:
            final_list.append(linelist[i])
    
    final_list=np.unique(final_list,axis=0).tolist()
    return final_list
        
#Read Image and convert into gray scale
srcimg = cv2.imread('Hough.png')
gimg=cv2.cvtColor(srcimg,cv2.COLOR_BGR2GRAY)


##################################################Draw Circles########################################################
cimg = cv2.Canny(gimg, 50, 100, None, 3, False)
circles = cv2.HoughCircles(cimg.T,cv2.HOUGH_GRADIENT,2,20,param1=500,param2=100,minRadius=18,maxRadius=60)

circles = np.uint16(np.around(circles))
with open ('results/coins.txt', 'w') as filehandler:
    for i in circles[0,:]:
        filehandler.write(str([i[0],i[1],i[2]])+'\n')
        cv2.circle(srcimg,(i[1],i[0]),i[2],(0,255,0),4)
        
cv2.imwrite("results/coins.jpg", srcimg)
#################################################Line Processing#####################################################
bsrcimg = cv2.imread('Hough.png')
gimg=  cv2.cvtColor(bsrcimg, cv2.COLOR_RGB2GRAY)
bimg = cv2.GaussianBlur(gimg, (5, 5), 1.5)
cimg = cv2.Canny(bimg, 100, 200)


houghmat,p,thetas=Hline_accum(cimg)


#######Get Hough Lines based on the length of the lines(votes)
hlines  = getHLines(houghmat, 30)
hlines1 = getHLines(houghmat, 1,148)
hlines2 = getHLines(houghmat, 8,78)
hlines3 = getHLines(houghmat, 127, 50)

hlines.extend(hlines1)
hlines.extend(hlines2)
hlines.extend([hlines3[-1]])

#################################################Cross Lines############################################################
filteredcl=[]
bsrcimg = cv2.imread('Hough.png')

for i in range(len(hlines)):
        theta = thetas[hlines[i][1]]
        
        if theta>-0.96 and theta<-0.92:
            filteredcl.append(hlines[i])

filteredcl=removerlaps(filteredcl,houghmat,mindist=10)
drawHLines(bsrcimg, filteredcl, p, thetas, (203,149,54),[5,7,9],'results/blue_lines.txt')           
cv2.imwrite("results/blue_lines.jpg", bsrcimg)

#################################################Vertical Lines#########################################################
filteredvl=[]
bsrcimg = cv2.imread('Hough.png')

for i in range(len(hlines)):
        theta = thetas[hlines[i][1]]
        if theta>-1.54 and theta<-1.51:
            filteredvl.append(hlines[i])

filteredvl=removerlaps(filteredvl,houghmat,mindist=10)

drawHLines(bsrcimg, filteredvl, p, thetas, (11,11,191), [5,7,8], 'results/red_lines.txt')
cv2.imwrite("results/red_lines.jpg", bsrcimg)
#######################################################################################################################

True